# 3-3,高阶API示范

下面的范例使用TensorFlow的高阶API实现线性回归模型。

TensorFlow的高阶API主要为tf.keras.models提供的模型的类接口。

使用Keras接口有以下3种方式构建模型：使用Sequential按层顺序构建模型，使用函数式API构建任意结构模型，继承Model基类构建自定义模型。

此处分别演示使用Sequential按层顺序构建模型以及继承Model基类构建自定义模型。

### 一，使用Sequential按层顺序构建模型【面向新手】

In [1]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers

#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

In [3]:
tf.keras.backend.clear_session()

linear = models.Sequential()
linear.add(layers.Dense(1, input_shape=(2,)))
linear.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [4]:
### 使用fit方法进行训练

linear.compile(optimizer="adam", loss="mse", metrics=['mae'])
linear.fit(X, Y, batch_size=20, epochs=200)

tf.print("w =", linear.layers[0].kernel)
tf.print("b =", linear.layers[0].bias)

Train on 800 samples
Epoch 1/200
800/800 [==============================] - 0s 256us/sample - loss: 120.5590 - mae: 9.3551
Epoch 2/200
800/800 [==============================] - 0s 38us/sample - loss: 115.2476 - mae: 9.1433
Epoch 3/200
800/800 [==============================] - 0s 38us/sample - loss: 110.2005 - mae: 8.9358
Epoch 4/200
800/800 [==============================] - 0s 39us/sample - loss: 105.3431 - mae: 8.7319
Epoch 5/200
800/800 [==============================] - 0s 38us/sample - loss: 100.7100 - mae: 8.5346
Epoch 6/200
800/800 [==============================] - 0s 37us/sample - loss: 96.2789 - mae: 8.3403
Epoch 7/200
800/800 [==============================] - 0s 39us/sample - loss: 92.0165 - mae: 8.1505
Epoch 8/200
800/800 [==============================] - 0s 40us/sample - loss: 87.9862 - mae: 7.9630
Epoch 9/200
800/800 [==============================] - 0s 36us/sample - loss: 84.0574 - mae: 7.7768
Epoch 10/200
800/800 [==============================] - 0s 36us/sample - 

800/800 [==============================] - 0s 36us/sample - loss: 4.5570 - mae: 1.6984
Epoch 83/200
800/800 [==============================] - 0s 36us/sample - loss: 4.5099 - mae: 1.6897
Epoch 84/200
800/800 [==============================] - 0s 36us/sample - loss: 4.4660 - mae: 1.6815
Epoch 85/200
800/800 [==============================] - 0s 36us/sample - loss: 4.4259 - mae: 1.6741
Epoch 86/200
800/800 [==============================] - 0s 33us/sample - loss: 4.3884 - mae: 1.6673
Epoch 87/200
800/800 [==============================] - 0s 29us/sample - loss: 4.3526 - mae: 1.6608
Epoch 88/200
800/800 [==============================] - 0s 32us/sample - loss: 4.3180 - mae: 1.6543
Epoch 89/200
800/800 [==============================] - 0s 34us/sample - loss: 4.2885 - mae: 1.6484
Epoch 90/200
800/800 [==============================] - 0s 34us/sample - loss: 4.2567 - mae: 1.6428
Epoch 91/200
800/800 [==============================] - 0s 46us/sample - loss: 4.2294 - mae: 1.6380
Epoch 92/200


800/800 [==============================] - 0s 37us/sample - loss: 3.7854 - mae: 1.5590
Epoch 164/200
800/800 [==============================] - 0s 37us/sample - loss: 3.7876 - mae: 1.5595
Epoch 165/200
800/800 [==============================] - 0s 36us/sample - loss: 3.7841 - mae: 1.5587
Epoch 166/200
800/800 [==============================] - 0s 37us/sample - loss: 3.7850 - mae: 1.5586
Epoch 167/200
800/800 [==============================] - 0s 38us/sample - loss: 3.7847 - mae: 1.5586
Epoch 168/200
800/800 [==============================] - 0s 36us/sample - loss: 3.7882 - mae: 1.5595
Epoch 169/200
800/800 [==============================] - 0s 36us/sample - loss: 3.7856 - mae: 1.5591
Epoch 170/200
800/800 [==============================] - 0s 37us/sample - loss: 3.7842 - mae: 1.5585
Epoch 171/200
800/800 [==============================] - 0s 36us/sample - loss: 3.7866 - mae: 1.5598
Epoch 172/200
800/800 [==============================] - 0s 36us/sample - loss: 3.7845 - mae: 1.5588
Epoc

### 二，继承Model基类构建自定义模型【面向专家】

In [5]:
import tensorflow as tf
from tensorflow.keras import models,layers,optimizers,losses,metrics


#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [8]:
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n,2],minval=-10,maxval=10) 
w0 = tf.constant([[2.0],[-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n,1],mean = 0.0,stddev= 2.0)  # @表示矩阵乘法,增加正态扰动

ds_train = tf.data.Dataset.from_tensor_slices((X[0:n*3//4,:], Y[0:n*3//4,:])) \
            .shuffle(buffer_size=1000).batch(20) \
            .prefetch(tf.data.experimental.AUTOTUNE) \
            .cache()

ds_valid = tf.data.Dataset.from_tensor_slices((X[n*3//4:,:],Y[n*3//4:,:])) \
             .shuffle(buffer_size = 1000).batch(20) \
             .prefetch(tf.data.experimental.AUTOTUNE) \
             .cache()

In [10]:
tf.keras.backend.clear_session()

class Mymodel(models.Model):
    def __init__(self):
        super(Mymodel, self).__init__()
        
    def build(self, input_shape):
        self.dense1 = layers.Dense(1)
        super(Mymodel, self).build(input_shape)
        
    def call(self, x):
        y = self.dense1(x)
        return (y)

model = Mymodel()
model.build(input_shape=(None, 2))
model.summary()

Model: "mymodel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [12]:
### 自定义训练循环(专家教程)

optimizer = optimizers.Adam()
loss_func = losses.MeanSquaredError()

train_loss = tf.keras.metrics.Mean(name="train_loss")
train_metric = tf.keras.metrics.MeanAbsoluteError(name="train_mae")

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_metric = tf.keras.metrics.MeanAbsoluteError(name='valid_mae')

@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)

@tf.function
def valid_step(model, features, labels):
    predictions = model(features)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)

@tf.function
def train_model(model, ds_train, ds_valid, epochs):
    for epoch in tf.range(1, epochs+1):
        for features, labels in ds_train:
            train_step(model, features, labels)
        
        for features, labels in ds_valid:
            valid_step(model, features, labels)
        
        logs = 'Epoch={},Loss:{},MAE:{},Valid Loss:{},Valid MAE:{}'
        
        if epoch % 100 == 0:
            printbar()
            tf.print(tf.strings.format(logs, 
                    (epoch, train_loss.result(), train_metric.result(), 
                     valid_loss.result(), valid_metric.result())))
            tf.print("w=", model.layers[0].kernel)
            tf.print("b=", model.layers[0].bias)
            tf.print("")
        
        train_loss.reset_states()
        train_metric.reset_states()
        valid_loss.reset_states()
        valid_metric.reset_states()
        
train_model(model, ds_train, ds_valid, 400)

================================================================================23:32:30
Epoch=100,Loss:53.222084,MAE:5.51857185,Valid Loss:56.7886276,Valid MAE:5.72988653
w= [[1.80102301]
 [-0.994384527]]
b= [1.97587514]

================================================================================23:32:36
Epoch=200,Loss:28.9310379,MAE:3.6355021,Valid Loss:30.4502754,Valid MAE:3.66744852
w= [[2.00816274]
 [-0.990700662]]
b= [2.94288826]

================================================================================23:32:42
Epoch=300,Loss:20.7498302,MAE:2.97957468,Valid Loss:21.5465202,Valid MAE:2.95705509
w= [[2.00814295]
 [-0.990829468]]
b= [3.00022626]

================================================================================23:32:48
Epoch=400,Loss:16.6697521,MAE:2.65235257,Valid Loss:17.1050091,Valid MAE:2.60282373
w= [[2.00815821]
 [-0.990836501]]
b= [3.00035906]

